In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

## make the dataset zipcode oriented

### load data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/ds340/nypd complaint data/1. adding zipcode to initial data/nypd_complaint_cleaned_zipcode.csv')

data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,PATROL_BORO,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,CMPLNT_FR_YEAR,CMPLNT_FR_MONTH,CMPLNT_FR_DAY,geometry,index_right,modzcta,label,zcta,pop_est,zipcode
0,0,279750914,12/20/2022,00:01:00,08/01/2023,12:00:00,71.0,12/31/2023,109,GRAND LARCENY,405.0,"LARCENY,GRAND BY THEFT OF CREDIT CARD",COMPLETED,FELONY,BROOKLYN,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,996766.0,178845.0,UNKNOWN,UNKNOWN,U,40.657557,-73.954888,"(40.657557, -73.954888)",PATROL BORO BKLYN SOUTH,UNKNOWN,UNKNOWN,F,2022,12,20,POINT (-73.954888 40.657557),112.0,11225.0,11225,11225,58882.0,11225.0
1,1,279714407,12/13/2022,09:00:00,12/14/2022,15:30:00,13.0,12/30/2023,109,GRAND LARCENY,458.0,"LARCENY,GRAND FROM RESIDENCE/BUILDING,UNATTEND...",COMPLETED,FELONY,MANHATTAN,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,991346.0,205375.0,UNKNOWN,UNKNOWN,U,40.730383,-73.974394,"(40.730383, -73.974394)",PATROL BORO MAN SOUTH,65+,WHITE,M,2022,12,13,POINT (-73.974394 40.730383),8.0,10009.0,10009,10009,57925.0,10009.0
2,2,279723001,12/15/2022,13:00:00,12/29/2023,14:00:00,62.0,12/29/2023,361,OFF. AGNST PUB ORD SENSBLTY &,639.0,AGGRAVATED HARASSMENT 2,COMPLETED,MISDEMEANOR,BROOKLYN,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,986958.0,160468.0,25-44,WHITE,F,40.607125,-73.990245,"(40.607125, -73.990245)",PATROL BORO BKLYN SOUTH,25-44,WHITE,F,2022,12,15,POINT (-73.990245 40.607125),102.0,11214.0,11214,11214,92946.0,11214.0
3,3,279694115,04/20/2022,12:00:00,09/24/2022,03:00:00,105.0,12/29/2023,341,PETIT LARCENY,349.0,"LARCENY,PETIT OF LICENSE PLATE",COMPLETED,MISDEMEANOR,QUEENS,FRONT OF,STREET,N.Y. POLICE DEPT,0,1053688.0,203981.0,(null),(null),(null),40.726287,-73.749476,"(40.726287, -73.749476)",PATROL BORO QUEENS SOUTH,45-64,ASIAN / PACIFIC ISLANDER,F,2022,4,20,POINT (-73.749476 40.726287),164.0,11427.0,11427,11427,24037.0,11427.0
4,4,279749489,03/20/2022,18:45:00,03/20/2022,18:50:00,94.0,12/29/2023,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BROOKLYN,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,997081.0,206903.0,25-44,WHITE,F,40.734570,-73.953698,"(40.73457, -73.953698)",PATROL BORO BKLYN NORTH,25-44,WHITE,M,2022,3,20,POINT (-73.953698 40.73457),109.0,11222.0,11222,11222,36492.0,11222.0


### preprocess

In [4]:
data['zipcode'] = data['zipcode'].astype(str).str.replace('.0', '')

### aggregate the data by 'zipcode'

In [6]:
aggregated_data = data.groupby('zipcode').agg(
    num_felony=pd.NamedAgg(column='LAW_CAT_CD', aggfunc=lambda x: (x == 'FELONY').sum()),
    num_violation=pd.NamedAgg(column='LAW_CAT_CD', aggfunc=lambda x: (x == 'VIOLATION').sum()),
    num_misdemeanor=pd.NamedAgg(column='LAW_CAT_CD', aggfunc=lambda x: (x == 'MISDEMEANOR').sum()),
    most_freq_ofns_desc=pd.NamedAgg(column='OFNS_DESC', aggfunc=lambda x: x.mode()[0]),
    most_freq_loc_occur = pd.NamedAgg(column='LOC_OF_OCCUR_DESC', aggfunc=lambda x: x.mode()[0]),
    most_freq_prem_type=pd.NamedAgg(column='PREM_TYP_DESC', aggfunc=lambda x: x.mode()[0]),
    most_freq_susp_age_group=pd.NamedAgg(column='SUSP_AGE_GROUP', aggfunc=lambda x: x.mode()[0]),
    most_freq_susp_race=pd.NamedAgg(column='SUSP_RACE', aggfunc=lambda x: x.mode()[0]),
    most_freq_susp_sex=pd.NamedAgg(column='SUSP_SEX', aggfunc=lambda x: x.mode()[0]),
    most_freq_vic_age_group=pd.NamedAgg(column='VIC_AGE_GROUP', aggfunc=lambda x: x.mode()[0]),
    most_freq_vic_race=pd.NamedAgg(column='VIC_RACE', aggfunc=lambda x: x.mode()[0]),
    most_freq_vic_sex=pd.NamedAgg(column='VIC_SEX', aggfunc=lambda x: x.mode()[0]),

)

aggregated_data['num_total_crimes'] = data.groupby('zipcode')['CMPLNT_NUM'].count()
aggregated_data.reset_index(inplace=True)

aggregated_data = aggregated_data.astype({
    'zipcode': str,
    'num_felony': int,
    'num_violation': int,
    'num_misdemeanor': int,
    'most_freq_ofns_desc': str,
    'most_freq_prem_type': str,
    'num_total_crimes': int
})

aggregated_data.head()

,zipcode,num_felony,num_violation,num_misdemeanor,most_freq_ofns_desc,most_freq_loc_occur,most_freq_prem_type,most_freq_susp_age_group,most_freq_susp_race,most_freq_susp_sex,most_freq_vic_age_group,most_freq_vic_race,most_freq_vic_sex,num_total_crimes
0,10001,2918,734,3677,PETIT LARCENY,INSIDE,STREET,UNKNOWN,BLACK,M,UNKNOWN,UNKNOWN,D,7329
1,10002,2278,1024,4381,PETIT LARCENY,INSIDE,STREET,UNKNOWN,UNKNOWN,M,UNKNOWN,UNKNOWN,M,7683
2,10003,1865,453,3066,PETIT LARCENY,INSIDE,STREET,UNKNOWN,UNKNOWN,M,UNKNOWN,UNKNOWN,D,5384
3,10004,143,70,229,PETIT LARCENY,INSIDE,STREET,UNKNOWN,UNKNOWN,M,UNKNOWN,UNKNOWN,M,442
4,10005,219,76,963,PETIT LARCENY,INSIDE,CHAIN STORE,UNKNOWN,BLACK,M,UNKNOWN,UNKNOWN,D,1258


## export the data

In [7]:
aggregated_data.to_csv('/content/drive/MyDrive/ds340/nypd complaint data/2. covert to zipcode oriented/zipcode_oriented.csv')